# Documents 테이블 img src 경로 변환

## 목표
- `src="approval_2025_img/apr15241300/4.jpg"` 
- → `src=\"/PMS_SITE-U7OI43JLDSMO/approval/attach/approval_2025_img/apr15241300/4.jpg\"`

## 변환 규칙
1. `src="` → `src=\"/PMS_SITE-U7OI43JLDSMO/approval/attach/`
2. 끝의 `"` → `\"`
3. 이미 `/PMS_SITE-U7OI43JLDSMO/`가 있는 경우는 제외

---
## 1. 라이브러리 및 DB 연결

In [1]:
import pymysql
import re
from collections import Counter

# DB 연결
conn = pymysql.connect(
    host='localhost',
    user='root',
    password='1234',
    database='any_approval',
    charset='utf8mb4'
)
cursor = conn.cursor(pymysql.cursors.DictCursor)

# img 태그 있는 문서 조회
cursor.execute("SELECT id, doc_body FROM documents WHERE doc_body LIKE '%<img%'")
documents = cursor.fetchall()

# 모든 src 값 추출
all_src = []
for doc in documents:
    body = doc['doc_body'] or ''
    # src="..." 또는 src=\"...\" 등 다양한 형태 매칭
    matches = re.findall(r'src=\\?["\']([^"\'<>]+)\\?["\']', body, re.IGNORECASE)
    for src in matches:
        all_src.append(src)

print(f"총 img src 개수: {len(all_src)}")
print("=" * 80)

# 경로 패턴 분석 (디렉토리 구조만 추출)
path_patterns = []
for src in all_src:
    # 파일명 제거하고 디렉토리 패턴만
    # 예: approval_2025_img/apr15241300/0.jpg → approval_2025_img/aprXXXXXX/
    pattern = re.sub(r'/apr\d+/', '/aprXXXXXX/', src)
    pattern = re.sub(r'/\d+\.jpg', '/X.jpg', pattern)
    pattern = re.sub(r'/\d+\.png', '/X.png', pattern)
    pattern = re.sub(r'/\d+\.gif', '/X.gif', pattern)
    path_patterns.append(pattern)

# 패턴별 개수
pattern_counts = Counter(path_patterns)
print("\n경로 패턴별 개수:")
print("-" * 80)
for pattern, count in pattern_counts.most_common():
    print(f"{count:5d}개 | {pattern}")

# 원본 경로 샘플도 보기
print("\n" + "=" * 80)
print("원본 src 샘플 (처음 30개):")
print("-" * 80)
for src in all_src[:30]:
    print(src)

cursor.close()
conn.close()

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [ ]:
import pymysql
import re
from collections import Counter
import copy

# DB 연결 설정
DB_CONFIG = {
    'host': 'localhost',
    'user': 'root',
    'password': 'ok',
    'database': 'any_approval',
    'charset': 'utf8mb4'
}

def get_connection():
    return pymysql.connect(**DB_CONFIG)

print("DB 설정 완료")

---
## 2. 현황 파악 - img 태그가 있는 문서 조회

In [ ]:
conn = get_connection()
cursor = conn.cursor(pymysql.cursors.DictCursor)

# img 태그가 있는 모든 문서 조회
cursor.execute("""
    SELECT id, doc_body 
    FROM documents 
    WHERE doc_body LIKE '%<img%'
""")

documents = cursor.fetchall()
print(f"img 태그가 있는 문서 수: {len(documents)}")

cursor.close()
conn.close()

---
## 3. 모든 img src 경로 패턴 분석

In [ ]:
# 모든 img 태그의 src 값 추출
all_src_values = []
img_pattern = re.compile(r'<img[^>]*\ssrc=(["\'])([^"\'>]+)\1', re.IGNORECASE)
img_pattern_escaped = re.compile(r'<img[^>]*\ssrc=\\(["\'])([^"\'>]+)\\\1', re.IGNORECASE)

for doc in documents:
    doc_body = doc['doc_body'] or ''
    
    # 일반 패턴: src="..." 또는 src='...'
    matches = img_pattern.findall(doc_body)
    for quote, src in matches:
        all_src_values.append(('normal', src, doc['id']))
    
    # 이스케이프된 패턴: src=\"...\" 또는 src=\'...\'
    matches_escaped = img_pattern_escaped.findall(doc_body)
    for quote, src in matches_escaped:
        all_src_values.append(('escaped', src, doc['id']))

print(f"총 img src 개수: {len(all_src_values)}")
print(f"  - 일반 따옴표 (normal): {len([x for x in all_src_values if x[0] == 'normal'])}")
print(f"  - 이스케이프된 따옴표 (escaped): {len([x for x in all_src_values if x[0] == 'escaped'])}")

In [ ]:
# 경로 패턴 분류
path_categories = {
    'already_pms': [],      # 이미 /PMS_SITE-U7OI43JLDSMO/ 포함
    'approval_img': [],     # approval_XXXX_img/ 패턴
    'other': []             # 기타 패턴
}

approval_img_pattern = re.compile(r'^approval_\d{4}_img/')

for quote_type, src, doc_id in all_src_values:
    if '/PMS_SITE-U7OI43JLDSMO/' in src:
        path_categories['already_pms'].append((quote_type, src, doc_id))
    elif approval_img_pattern.match(src):
        path_categories['approval_img'].append((quote_type, src, doc_id))
    else:
        path_categories['other'].append((quote_type, src, doc_id))

print("=" * 60)
print("경로 패턴별 분류")
print("=" * 60)
print(f"이미 PMS 경로 포함: {len(path_categories['already_pms'])}개")
print(f"approval_XXXX_img 패턴: {len(path_categories['approval_img'])}개")
print(f"기타 패턴: {len(path_categories['other'])}개")
print("=" * 60)

In [ ]:
# 기타 패턴 상세 확인 (있다면)
if path_categories['other']:
    print("⚠️  기타 패턴 목록 (확인 필요):")
    print("-" * 60)
    for quote_type, src, doc_id in path_categories['other'][:30]:  # 최대 30개만
        print(f"  [ID: {doc_id}] [{quote_type}] {src}")
    if len(path_categories['other']) > 30:
        print(f"  ... 외 {len(path_categories['other']) - 30}개 더 있음")
else:
    print("✅ 기타 패턴 없음 - 모든 경로가 예상 패턴과 일치")

In [ ]:
# 이미 PMS 경로가 있는 것 샘플 확인
if path_categories['already_pms']:
    print("ℹ️  이미 PMS 경로가 포함된 것 샘플:")
    print("-" * 60)
    for quote_type, src, doc_id in path_categories['already_pms'][:10]:
        print(f"  [ID: {doc_id}] [{quote_type}] {src}")
else:
    print("✅ 이미 PMS 경로가 있는 것 없음")

In [ ]:
# 연도별 분포 확인
year_pattern = re.compile(r'approval_(\d{4})_img')
years = []

for quote_type, src, doc_id in path_categories['approval_img']:
    match = year_pattern.search(src)
    if match:
        years.append(match.group(1))

year_counts = Counter(years)
print("연도별 분포:")
for year, count in sorted(year_counts.items()):
    print(f"  {year}: {count}개")

---
## 4. 변환 대상 확정 및 요약

In [ ]:
# 변환 대상: normal 따옴표이면서 approval_img 패턴이거나 other 패턴
# (이미 escaped된 것과 이미 PMS 경로가 있는 것은 제외)

to_transform_approval = [x for x in path_categories['approval_img'] if x[0] == 'normal']
to_transform_other = [x for x in path_categories['other'] if x[0] == 'normal']
already_escaped = [x for x in all_src_values if x[0] == 'escaped']
already_pms_normal = [x for x in path_categories['already_pms'] if x[0] == 'normal']

print("=" * 60)
print("변환 대상 요약")
print("=" * 60)
print(f"🔄 변환 대상 (approval_img 패턴, normal 따옴표): {len(to_transform_approval)}개")
print(f"🔄 변환 대상 (기타 패턴, normal 따옴표): {len(to_transform_other)}개")
print(f"⏭️  제외 (이미 escaped): {len(already_escaped)}개")
print(f"⏭️  제외 (이미 PMS 경로, normal): {len(already_pms_normal)}개")
print("=" * 60)
print(f"총 변환 대상: {len(to_transform_approval) + len(to_transform_other)}개")
print("=" * 60)

---
## 5. 변환 함수 정의

In [ ]:
def transform_img_src(doc_body):
    """
    img 태그의 src 속성을 변환
    - src="path" → src=\"/PMS_SITE-U7OI43JLDSMO/approval/attach/path\"
    - 이미 /PMS_SITE-U7OI43JLDSMO/가 있는 경로는 제외
    - 이미 이스케이프된 것(src=\")은 제외
    """
    if not doc_body:
        return doc_body, 0
    
    transform_count = 0
    
    def replace_src(match):
        nonlocal transform_count
        full_match = match.group(0)
        src_value = match.group(1)
        
        # 이미 PMS 경로가 있으면 스킵
        if '/PMS_SITE-U7OI43JLDSMO/' in src_value:
            return full_match
        
        transform_count += 1
        new_src = f'/PMS_SITE-U7OI43JLDSMO/approval/attach/{src_value}'
        return f'src=\"{new_src}\"'
    
    # src="..." 패턴만 매칭 (src=\"...\"는 매칭 안됨)
    # negative lookbehind로 앞에 \가 없는 경우만 매칭
    pattern = r'(?<!\\)src="([^"]+)"'
    
    new_body = re.sub(pattern, replace_src, doc_body)
    
    return new_body, transform_count

print("변환 함수 정의 완료")

In [ ]:
# 변환 함수 테스트
test_cases = [
    '<img alt="" src="approval_2025_img/apr15241300/0.jpg" style="width: 790px">',
    '<img src="approval_2020_img/apr2978801/0.jpg"/>',
    '<img src=\"/PMS_SITE-U7OI43JLDSMO/approval/attach/approval_2020_img/apr123/0.jpg\"/>',  # 이미 변환된 것
    '<p>텍스트</p><img src="approval_2015_img/apr111/1.jpg"><img src="approval_2010_img/apr222/2.jpg">',  # 여러 개
]

print("변환 테스트:")
print("=" * 80)
for test in test_cases:
    result, count = transform_img_src(test)
    print(f"원본: {test}")
    print(f"결과: {result}")
    print(f"변환 수: {count}")
    print("-" * 80)

---
## 6. 변환 전 상태 저장 (검증용)

In [ ]:
# 변환 전 상태 저장
before_state = {
    'total_docs_with_img': len(documents),
    'total_img_tags': len(all_src_values),
    'normal_quotes': len([x for x in all_src_values if x[0] == 'normal']),
    'escaped_quotes': len([x for x in all_src_values if x[0] == 'escaped']),
    'already_pms': len(path_categories['already_pms']),
    'to_transform': len(to_transform_approval) + len(to_transform_other),
}

print("변환 전 상태 저장:")
for key, value in before_state.items():
    print(f"  {key}: {value}")

---
## 7. 변환 실행 (DRY RUN - 실제 DB 변경 없음)

In [ ]:
# DRY RUN: 실제 변환 결과 미리보기
dry_run_results = []
total_transform_count = 0

for doc in documents:
    doc_id = doc['id']
    original_body = doc['doc_body'] or ''
    
    new_body, count = transform_img_src(original_body)
    
    if count > 0:
        dry_run_results.append({
            'id': doc_id,
            'original': original_body,
            'transformed': new_body,
            'count': count
        })
        total_transform_count += count

print(f"DRY RUN 결과:")
print(f"  변환된 문서 수: {len(dry_run_results)}")
print(f"  총 변환된 img 태그 수: {total_transform_count}")
print(f"  예상 변환 수와 일치 여부: {total_transform_count == before_state['to_transform']}")

In [ ]:
# 변환 샘플 확인
print("변환 샘플 (처음 5개):")
print("=" * 80)
for result in dry_run_results[:5]:
    print(f"문서 ID: {result['id']}")
    print(f"변환 수: {result['count']}")
    
    # img 태그 부분만 추출해서 보여주기
    orig_imgs = re.findall(r'<img[^>]+>', result['original'])
    trans_imgs = re.findall(r'<img[^>]+>', result['transformed'])
    
    for i, (orig, trans) in enumerate(zip(orig_imgs, trans_imgs)):
        if orig != trans:
            print(f"  원본: {orig[:100]}..." if len(orig) > 100 else f"  원본: {orig}")
            print(f"  변환: {trans[:100]}..." if len(trans) > 100 else f"  변환: {trans}")
    print("-" * 80)

---
## 8. 검증 함수 정의

In [ ]:
def verify_transformation(documents_after):
    """
    변환 후 검증 수행
    """
    errors = []
    warnings = []
    
    # 새로운 상태 계산
    all_src_after = []
    img_pattern = re.compile(r'<img[^>]*\ssrc=(["\'])([^"\'>]+)\1', re.IGNORECASE)
    img_pattern_escaped = re.compile(r'<img[^>]*\ssrc=\\(["\'])([^"\'>]+)\\\1', re.IGNORECASE)
    
    for doc in documents_after:
        doc_body = doc['doc_body'] or ''
        
        matches = img_pattern.findall(doc_body)
        for quote, src in matches:
            all_src_after.append(('normal', src, doc['id']))
        
        matches_escaped = img_pattern_escaped.findall(doc_body)
        for quote, src in matches_escaped:
            all_src_after.append(('escaped', src, doc['id']))
    
    after_state = {
        'total_docs_with_img': len(documents_after),
        'total_img_tags': len(all_src_after),
        'normal_quotes': len([x for x in all_src_after if x[0] == 'normal']),
        'escaped_quotes': len([x for x in all_src_after if x[0] == 'escaped']),
    }
    
    # 검증 1: 총 img 태그 수 동일
    if after_state['total_img_tags'] != before_state['total_img_tags']:
        errors.append(f"❌ img 태그 수 불일치: 전 {before_state['total_img_tags']} → 후 {after_state['total_img_tags']}")
    else:
        print(f"✅ img 태그 수 일치: {after_state['total_img_tags']}")
    
    # 검증 2: escaped 따옴표 수 증가량 확인
    expected_escaped = before_state['escaped_quotes'] + before_state['to_transform']
    if after_state['escaped_quotes'] != expected_escaped:
        errors.append(f"❌ escaped 따옴표 수 불일치: 예상 {expected_escaped} → 실제 {after_state['escaped_quotes']}")
    else:
        print(f"✅ escaped 따옴표 수 정확: {after_state['escaped_quotes']}")
    
    # 검증 3: normal 따옴표 수 감소량 확인
    expected_normal = before_state['normal_quotes'] - before_state['to_transform']
    if after_state['normal_quotes'] != expected_normal:
        errors.append(f"❌ normal 따옴표 수 불일치: 예상 {expected_normal} → 실제 {after_state['normal_quotes']}")
    else:
        print(f"✅ normal 따옴표 수 정확: {after_state['normal_quotes']}")
    
    # 검증 4: 모든 approval_img 경로에 PMS 접두사 있는지 확인
    missing_pms = []
    for quote_type, src, doc_id in all_src_after:
        if re.match(r'^approval_\d{4}_img/', src):
            missing_pms.append((doc_id, src))
    
    if missing_pms:
        errors.append(f"❌ PMS 접두사 누락: {len(missing_pms)}개")
        for doc_id, src in missing_pms[:5]:
            print(f"   - [ID: {doc_id}] {src}")
    else:
        print(f"✅ 모든 approval_img 경로에 PMS 접두사 포함")
    
    # 검증 5: 중복 변환 확인 (//PMS_SITE 등)
    double_transform = []
    for doc in documents_after:
        doc_body = doc['doc_body'] or ''
        if '/PMS_SITE-U7OI43JLDSMO//PMS_SITE-U7OI43JLDSMO/' in doc_body or \
           '/PMS_SITE-U7OI43JLDSMO/approval/attach//PMS_SITE-U7OI43JLDSMO/' in doc_body:
            double_transform.append(doc['id'])
    
    if double_transform:
        errors.append(f"❌ 중복 변환 발견: {len(double_transform)}개")
    else:
        print(f"✅ 중복 변환 없음")
    
    return errors, warnings, after_state

print("검증 함수 정의 완료")

---
## 9. DRY RUN 검증

In [ ]:
# DRY RUN 결과로 검증 수행
# 변환된 문서 + 변환 안된 문서를 합쳐서 전체 상태 만들기

transformed_ids = {r['id'] for r in dry_run_results}

documents_after_dry = []
for doc in documents:
    if doc['id'] in transformed_ids:
        # 변환된 버전 사용
        for r in dry_run_results:
            if r['id'] == doc['id']:
                documents_after_dry.append({'id': doc['id'], 'doc_body': r['transformed']})
                break
    else:
        # 원본 그대로
        documents_after_dry.append(doc)

print("DRY RUN 검증 수행:")
print("=" * 60)
errors, warnings, after_state = verify_transformation(documents_after_dry)

if errors:
    print("\n❌ 검증 실패:")
    for error in errors:
        print(f"  {error}")
else:
    print("\n✅ 모든 검증 통과!")

---
## 10. 실제 DB 변환 실행

⚠️ **주의: 이 셀을 실행하면 실제 DB가 변경됩니다!**

In [ ]:
# 실행 전 최종 확인
print("=" * 60)
print("실제 DB 변환 실행 전 최종 확인")
print("=" * 60)
print(f"변환 대상 문서 수: {len(dry_run_results)}")
print(f"변환 대상 img 태그 수: {total_transform_count}")
print(f"DRY RUN 검증 결과: {'통과' if not errors else '실패'}")
print("=" * 60)

if errors:
    print("\n❌ DRY RUN 검증 실패로 실행할 수 없습니다.")
else:
    print("\n다음 셀을 실행하면 실제 DB가 변경됩니다.")

In [ ]:
# ⚠️ 실제 DB 변환 실행
# 이 셀을 실행하려면 아래 주석을 해제하세요

EXECUTE_FOR_REAL = False  # True로 변경하면 실행됨

if EXECUTE_FOR_REAL and not errors:
    conn = get_connection()
    cursor = conn.cursor()
    
    try:
        updated_count = 0
        
        for result in dry_run_results:
            cursor.execute(
                "UPDATE documents SET doc_body = %s WHERE id = %s",
                (result['transformed'], result['id'])
            )
            updated_count += 1
            
            if updated_count % 100 == 0:
                print(f"진행 중: {updated_count}/{len(dry_run_results)}")
        
        # 커밋 전 최종 확인
        print(f"\n총 {updated_count}개 문서 UPDATE 완료 (아직 커밋 안됨)")
        
        # 변환 후 검증
        cursor_verify = conn.cursor(pymysql.cursors.DictCursor)
        cursor_verify.execute("""
            SELECT id, doc_body 
            FROM documents 
            WHERE doc_body LIKE '%<img%'
        """)
        documents_after_real = cursor_verify.fetchall()
        cursor_verify.close()
        
        print("\n변환 후 검증 수행:")
        print("=" * 60)
        errors_after, warnings_after, state_after = verify_transformation(documents_after_real)
        
        if errors_after:
            print("\n❌ 검증 실패! 롤백합니다.")
            conn.rollback()
            for error in errors_after:
                print(f"  {error}")
        else:
            print("\n✅ 검증 통과! 커밋합니다.")
            conn.commit()
            print(f"✅ {updated_count}개 문서 변환 완료!")
        
    except Exception as e:
        print(f"❌ 오류 발생: {e}")
        conn.rollback()
        raise
    finally:
        cursor.close()
        conn.close()
else:
    if not EXECUTE_FOR_REAL:
        print("EXECUTE_FOR_REAL = True로 변경하면 실행됩니다.")
    elif errors:
        print("DRY RUN 검증 실패로 실행할 수 없습니다.")

---
## 11. 변환 후 최종 확인

In [ ]:
# 변환 완료 후 최종 확인 쿼리
# 실제 변환 후 이 셀을 실행하세요

VERIFY_AFTER_COMMIT = False  # 변환 후 True로 변경

if VERIFY_AFTER_COMMIT:
    conn = get_connection()
    cursor = conn.cursor(pymysql.cursors.DictCursor)
    
    cursor.execute("""
        SELECT id, doc_body 
        FROM documents 
        WHERE doc_body LIKE '%<img%'
    """)
    
    final_documents = cursor.fetchall()
    
    print("최종 상태 확인:")
    print("=" * 60)
    errors_final, warnings_final, state_final = verify_transformation(final_documents)
    
    print("\n최종 상태 요약:")
    for key, value in state_final.items():
        print(f"  {key}: {value}")
    
    cursor.close()
    conn.close()